<img src="../img/GTK_Logo_Social Icon.jpg" width=175 align="right" />

# Worksheet 5.2: DGA Detection - Answers
This worksheet covers concepts covered in the second part of Module 5 - Supervised Learning.  It should take no more than 40-60 minutes to complete.  Please raise your hand if you get stuck.  

## Import the Libraries
For this exercise, we will be using:
* Pandas (http://pandas.pydata.org/pandas-docs/stable/)
* Numpy (https://docs.scipy.org/doc/numpy/reference/)
* Matplotlib (http://matplotlib.org/api/pyplot_api.html)
* Scikit-learn (http://scikit-learn.org/stable/documentation.html)
* YellowBrick (http://www.scikit-yb.org/en/latest/)
* Seaborn (https://seaborn.pydata.org)
* Lime (https://github.com/marcotcr/lime)

In [ ]:
# Load Libraries - Make sure to run this cell!
import pandas as pd
import numpy as np
import re
from collections import Counter
from sklearn import feature_extraction, tree, model_selection, metrics
from sklearn.preprocessing import LabelEncoder
from yellowbrick.classifier import ClassificationReport
from yellowbrick.classifier import ConfusionMatrix
import matplotlib.pyplot as plt
import matplotlib
import lime
import io
import pickle
%matplotlib inline

from six.moves import cPickle as pickle

import warnings
warnings.filterwarnings('ignore')

## Worksheet - DGA Detection using Machine Learning

This worksheet is a step-by-step guide on how to detect domains that were generated using "Domain Generation Algorithm" (DGA). We will walk you through the process of transforming raw domain strings to Machine Learning features and creating a decision tree classifer which you will use to determine whether a given domain is legit or not. Once you have implemented the classifier, the worksheet will walk you through evaluating your model.  

Overview 2 main steps:

1. **Feature Engineering** - from raw domain strings to numeric Machine Learning features using DataFrame manipulations
2. **Machine Learning Classification** - predict whether a domain is legit or not using a Decision Tree Classifier


  

**DGA - Background**

"Various families of malware use domain generation
algorithms (DGAs) to generate a large number of pseudo-random
domain names to connect to a command and control (C2) server.
In order to block DGA C2 traffic, security organizations must
first discover the algorithm by reverse engineering malware
samples, then generate a list of domains for a given seed. The
domains are then either preregistered, sink-holed or published
in a DNS blacklist. This process is not only tedious, but can
be readily circumvented by malware authors. An alternative
approach to stop malware from using DGAs is to intercept DNS
queries on a network and predict whether domains are DGA
generated. Much of the previous work in DGA detection is based
on finding groupings of like domains and using their statistical
properties to determine if they are DGA generated. However,
these techniques are run over large time windows and cannot be
used for real-time detection and prevention. In addition, many of
these techniques also use contextual information such as passive
DNS and aggregations of all NXDomains throughout a network.
Such requirements are not only costly to integrate, they may not
be possible due to real-world constraints of many systems (such
as endpoint detection). An alternative to these systems is a much
harder problem: detect DGA generation on a per domain basis
with no information except for the domain name. Previous work
to solve this harder problem exhibits poor performance and many
of these systems rely heavily on manual creation of features;
a time consuming process that can easily be circumvented by
malware authors..."    
[Citation: Woodbridge et. al 2016: "Predicting Domain Generation Algorithms with Long Short-Term Memory Networks"]

A better alternative for real-world deployment would be to use "featureless deep learning" - We have a separate notebook where you can see how this can be implemented!

**However, let's learn the basics first!!!**


## Feature Engineering

#### Breakpoint: Load Features and Labels

If you got stuck in Part 1, please simply load the feature matrix we prepared for you, so you can move on to Part 2 and train a Decision Tree Classifier.

In [ ]:
#load full dataset
df_final = pd.read_csv('../data/dga_features_final_df.csv')

#If you didn't get a working dataset, uncomment this line
#df_final = pd.read_csv('../data/our_data_dga_features_final_df.csv')


print(df_final['isDGA'].value_counts())
df_final.head()

## Machine Learning - Supervised Learning

To learn simple classification procedures using [sklearn](http://scikit-learn.org/stable/) we have split the work flow into 5 steps.

# Prepare Feature matrix and ```target``` vector containing the URL labels

- In statistics, the feature matrix is often referred to as ```X```
- target is a vector containing the labels for each URL (often also called ```y``` in statistics)
- In sklearn both the input and target can either be a pandas DataFrame/Series or numpy array/vector respectively (can't be lists!)

Tasks:
- 1.1 assign 'isDGA' column to a pandas Series named 'target'
- 1.2 encode the strings as digits using sklearn ```LabelEncoder``` 
- 1.3 drop 'isDGA' column from ```df_final``` DataFrame and name the resulting pandas DataFrame ```feature_matrix```

## 1.1 Create a variable named 'target' 
that contains whether the row was dga or legit

In [ ]:
target = df_final['isDGA']
target.head()

## 1.2 Encode strings as digits in the target variable

LabelEncoder is an encoder from sklearn that will transform a target vector (not the features) into integers.

[label encoder doc](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.LabelEncoder.html)


In [ ]:
label_encoder_model = LabelEncoder()
encoded_target = label_encoder_model.fit_transform(target)

# print our classes with the .classes_ method
label_encoder_model.classes_

In [ ]:
#look at encoded targets
encoded_target

The reason we like to encode targets using sklearn, is that once we are finished modeling, we can use the same model to turn it back into strings. This is really useful when plotting metrics or looking at the results (especially when you have more than one class

In [ ]:
# transform our encoded targets back into strings
label_encoder_model.inverse_transform(encoded_target)

## 1.3 Create the Feature Matrix

In order to train a model you have to separate the features from the target in to separate objects. Drop **isDGA** column from ```df_final``` DataFrame and name the resulting pandas DataFrame ```feature_matrix```

In [ ]:
feature_matrix = df_final.drop(['isDGA'], axis='columns')
feature_matrix.head()

In [ ]:
# Creata a list of our feature names for plotting later and if we need to pull the features again from the full dataframe.
feature_names = feature_matrix.columns.to_list()
print(feature_names)

# Test-Train split

Tasks:
- split your `feature_matrix` and `target` vector into **train** and **test** subsets using sklearn [model_selection.train_test_split](http://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html)

Output of the split should be 2 complete sets of data separated by features and labels: 
 - feature_matrix_train: 75% of the feature matrix (data)
 - feature_matrix_test: the remaining 25% of the feature matrix
 - target_train: the labels for the train features
 - target_test: the labels for the test features

In [ ]:
# Split the data set into training and test data
feature_matrix_train, feature_matrix_test, target_train, target_test = (model_selection.
                                                                        train_test_split(
                                                                           feature_matrix, 
                                                                           encoded_target,
                                                                           test_size=0.25, 
                                                                           random_state=33))

In [ ]:
feature_matrix_train.shape

In [ ]:
feature_matrix_test.shape

In [ ]:
target_train[1:5]

In [ ]:
target_test[1:5]

# Train the model and make a prediction

Finally, we have prepared and segmented the data. Let's start classifying!!   

Tasks:

-  Use the sklearn [tree.DecisionTreeClassfier()](http://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeClassifier.html), create a decision tree with default parameters, and train it using the ```.fit()``` function with ```feature_matrix_train``` and ```target_train``` data.
-  Next, pull a few random rows from the data and see if your classifier got it correct.

In [ ]:
d_tree_model = tree.DecisionTreeClassifier()  
d_tree_model = d_tree_model.fit(feature_matrix_train, target_train)

That's it! You trained the model. Now extract a row from the test set to see if the model can predict the correct answer by comparing it to the test target (ground truth). 

In [ ]:
# Extract a row from the test data

row_number = 14
test_feature = feature_matrix_test[row_number:row_number+1]

# Make the prediction
pred = d_tree_model.predict(test_feature)

#Let's go back to having strings in the targets to compare 
#b/c it'e easier to read. We can do this using the inverse_transform from the label encoder model
pred_string = label_encoder_model.inverse_transform(pred)

# pull out the ground truth for this row
test_target = target_test[row_number:row_number+1]
# transfrom to a string
test_target_string = label_encoder_model.inverse_transform(test_target)

                                                    
# print the results and the ground truth
print('Predicted class:', pred_string)
print('Ground truth class:', test_target_string)
print('Accurate prediction?', pred_string == test_target_string)

# Assess model performance

- Make predictions for all your data. Call the ```.predict()``` method on the model ```d_tree_model``` with your test data ```feature_matrix_test``` and store the results in a variable called ```target_pred```. This will be data that the model has not 'seen' before so we will use these predictions to evaluate how well the model can predict the correct answer on new data by calling a few different metrics functions from our libraries like yellowbrick and sklearn.

- Then calculate the accuracy using ```target_test``` (which are the true labels/ground truth) AND your models predictions on the test portion ```target_pred``` as inputs. The advantage here is to see how your model performs on new data it has not seen during the training phase. 

In [ ]:
# make predictions on all of the test data
target_pred = d_tree_model.predict(feature_matrix_test)

# print a sample of the predictions
print(target_pred[0:5])

- Use the label_encoder_model from above to create a transformed version of these **target_pred** results and save as **transformed_test_predictions**. The transformation will take the data from from a vector of 0s and 1s back into a vector of strings ('dga', 'legit'). We do this because it will be much easier to interpret in our metrics and in any plots. 

- Creat a transformed version of the **target_test** data (ground truth) **transformed_target_test** so that when we compare the predictions against the truth it's in the same format. 

In [ ]:
transformed_test_predictions = label_encoder_model.inverse_transform(target_pred)
transformed_target_test = label_encoder_model.inverse_transform(target_test)

# print a sample
print(transformed_target_test[0:5])

- Use sklearn [metrics.accuracy_score](http://scikit-learn.org/stable/modules/generated/sklearn.metrics.accuracy_score.html) to determine your model accuracy. 

In [ ]:
print("Accuracy:", metrics.accuracy_score(transformed_target_test, transformed_test_predictions))

- Print out the confusion matrix using [metrics.confusion_matrix](http://scikit-learn.org/stable/modules/generated/sklearn.metrics.confusion_matrix.html)

In [ ]:
print('Confusion Matrix\n', metrics.confusion_matrix(transformed_target_test, transformed_test_predictions))

In [ ]:
# Classification Report...neat summary
print(metrics.classification_report(transformed_target_test, transformed_test_predictions))

Use Yellowbrick's Classification Report and ConfusionMatrix function to visualize the classification report and confusion matrix. (https://www.scikit-yb.org/en/latest/)

In [ ]:
viz = ConfusionMatrix(d_tree_model)
viz.fit(feature_matrix_train, target_train)
viz.score(feature_matrix_test, target_test)
viz.show()

In [ ]:
viz = ClassificationReport(d_tree_model)
viz.fit(feature_matrix_train, target_train)
viz.score(feature_matrix_test, target_test)
viz.show()

#### (Optional) Visualizing your Tree
As an optional step, you can actually visualize your tree.  The following code will generate a graph of your decision tree.  You will need graphviz (http://www.graphviz.org) and pydotplus (or pydot) installed for this to work.
The Griffon VM has this installed already, but if you try this on a Mac, or Linux machine you will need to install graphviz.

In [ ]:
# These libraries are used to visualize the decision tree and require that you have GraphViz
# and pydot or pydotplus installed on your computer.

from IPython.core.display import Image
import pydotplus as pydot


dot_data = io.StringIO() 
tree.export_graphviz(d_tree_model, out_file=dot_data, 
                     feature_names=feature_names,
                    filled=True, rounded=True,  
                    special_characters=True) 

graph = pydot.graph_from_dot_data(dot_data.getvalue()) 
Image(graph.create_png())


## Other Models
Now that you've built a Decision Tree, let's try out two other classifiers and see how they perform on this data.  For this next exercise, create classifiers using:

* Support Vector Machine
* Random Forest
* K-Nearest Neighbors (http://scikit-learn.org/stable/modules/generated/sklearn.neighbors.KNeighborsClassifier.html)  

Once you've done that, run the various performance metrics to determine which classifier works best.

In [ ]:
from sklearn import svm
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier


In [ ]:
#Create the Random Forest Classifier
random_forest_clf = RandomForestClassifier(n_estimators=10, 
                             max_depth=None, 
                             min_samples_split=2, 
                             random_state=0)

random_forest_clf = random_forest_clf.fit(feature_matrix_train, target_train)

In [ ]:
random_forest_predictions = random_forest_clf.predict(feature_matrix_test)

In [ ]:
viz = ClassificationReport(random_forest_clf)
viz.fit(feature_matrix_train, target_train)
viz.score(feature_matrix_test, target_test)
viz.poof()

In [ ]:
metrics.accuracy_score(target_test, random_forest_predictions)

In [ ]:
#Next, create the SVM classifier
svm_classifier = svm.SVC()
svm_classifier = svm_classifier.fit(feature_matrix_train, target_train)  

In [ ]:
svm_preds = svm_classifier.predict(feature_matrix_test)

In [ ]:
viz = ClassificationReport(svm_classifier)
viz.fit(feature_matrix_train, target_train)
viz.score(feature_matrix_test, target_test)
viz.poof()

In [ ]:
#Finally the knn
knn_clf = KNeighborsClassifier()
knn_clf = knn_clf.fit(feature_matrix_train, target_train) 

In [ ]:
viz = ClassificationReport(knn_clf)
viz.fit(feature_matrix_train, target_train)
viz.score(feature_matrix_test, target_test)
viz.poof()

## Explain a Prediction
In the example below, you can use LIME to explain how a classifier arrived at its prediction.  Try running LIME with the various classifiers you've created and various rows to see how it functions. 

In [ ]:
import lime.lime_tabular
explainer = lime.lime_tabular.LimeTabularExplainer(feature_matrix_train, 
                                                   feature_names=feature_names, 
                                                   class_names=['legit', 'isDGA'], 
                                                   discretize_continuous=False)

In [ ]:
exp = explainer.explain_instance(feature_matrix_test.iloc[12], 
                                 d_tree_model.predict_proba, 
                                 num_features=6)

In [ ]:
exp.show_in_notebook(show_table=True, show_all=True)

In [ ]:
feature_matrix_test.iloc[8]

In [ ]:
# Save the Decision Tree for Worksheet 11
filename = '../data/dga_decision_tree.sav'
pickle.dump(clf, open(filename, 'wb'))